In [3]:
from pyspark.sql import SparkSession
from pyspark.sql import functions 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, from_json, window, count,trunc,to_timestamp,date_trunc

import pandas as pd
import os
import json
# from dotenv import load_dotenv

In [4]:

MINIO_ENDPOINT = "http://minio:9000"
HIVE_METASTORE_URI = "thrift://docker-compose-hive-metastore-1:9083" 

# Credentials (ต้องตรงกับ docker-compose)
ACCESS_KEY = "minio"
SECRET_KEY = "minio123"


spark = (
    SparkSession.builder
    .appName("test-lmwn")

    # s3
    # .config("spark.hadoop.fs.s3a.endpoint", "http://s3:9000")
    # .config("spark.hadoop.fs.s3a.access.key", "minioadmin")
    # .config("spark.hadoop.fs.s3a.secret.key", "miniopassword")
    # .config("spark.hadoop.fs.s3a.path.style.access", "true")
    # .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    # .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")

    # --- S3 (MinIO) Config ---
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")



    
    # # iceberg
    # .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
    # .config("spark.sql.catalog.iceberg.type", "hive")
    # .config("spark.sql.catalog.iceberg.uri", HIVE_METASTORE_URI)
    # .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/iceberg_data")

   # --- Iceberg Catalog Config (เชื่อม Hive) ---
    # เปลี่ยนคำว่า iceberg เป็น hive ให้หมด
    .config("spark.sql.catalog.hive", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.hive.type", "hive")
    .config("spark.sql.catalog.hive.uri", HIVE_METASTORE_URI)
    .config("spark.sql.catalog.hive.warehouse", "s3a://warehouse/users2")
    




    
    # kafka
    
    .config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0"
    )

    .getOrCreate()
)


/usr/local/lib/python3.8/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-16a07e74-22bf-4d0c-92f1-71cdc89e7805;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 389ms :: artifacts dl 14ms
	:: modules in us

In [55]:
%pip install ipython-sql sqlalchemy trino[sqlalchemy]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [ipython-sql]

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [60]:
pip install trino pandas


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [61]:
# 1. โหลด Extension
%load_ext sql

# 2. เชื่อมต่อ (อย่าลืมเช็ค Host ว่าเป็น localhost หรือ docker-name)
# กรณีรัน Jupyter ใน Docker เดียวกัน:
%sql trino://admin@docker-compose-trino-coordinator-1:8080/iceberg/test_db

# กรณีรัน Jupyter บน Windows/Mac (ข้างนอก):
# %sql trino://admin@localhost:8080/iceberg/test_db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [63]:
# 1. โหลด Extension
%load_ext sql

# --- 🛠️ FIX: แก้ Error 'DEFAULT' Style ไม่เจอ ---
%config SqlMagic.style = 'PLAIN_COLUMNS'
# ---------------------------------------------

# 2. เชื่อมต่อ Trino (ถ้า Jupyter อยู่ใน Docker Network เดียวกัน)
%sql trino://admin@docker-compose-trino-coordinator-1:8080/iceberg/test_db

# 3. รัน Query (ดึงจำนวนจากตารางที่ Spark สร้างไว้)
# หมายเหตุ: ต้องมั่นใจว่า Spark สร้างตาราง 'jupyter_users' ใน 'demo_schema' เสร็จแล้วนะ
%sql SELECT count(*) FROM iceberg.demo_schema.jupyter_users

The sql extension is already loaded. To reload it, use:
  %reload_ext sql
 * trino://admin@docker-compose-trino-coordinator-1:8080/iceberg/test_db
Done.


KeyError: 'PLAIN_COLUMNS'

In [66]:
import pandas as pd
from sqlalchemy import create_engine

# 1. สร้าง Connection Engine
# เปลี่ยน host เป็น localhost ถ้า Jupyter รันนอก Docker
engine = create_engine('trino://admin@docker-compose-trino-coordinator-1:8080/iceberg/test_db')

# 2. เขียน SQL
sql = "SELECT *  total_count FROM iceberg.demo_schema.jupyter_users"

# 3. ดึงข้อมูล
try:
    print("⏳ Querying data...")
    df = pd.read_sql(sql, engine)
    print("✅ Result:")
    print(df)
except Exception as e:
    print(f"❌ Error: {e}")
finally:
    engine.dispose()

⏳ Querying data...
❌ Error: (trino.exceptions.TrinoUserError) TrinoUserError(type=USER_ERROR, name=SYNTAX_ERROR, message="line 1:11: mismatched input 'total_count'. Expecting: ',', 'EXCEPT', 'FETCH', 'FROM', 'GROUP', 'HAVING', 'INTERSECT', 'LIMIT', 'OFFSET', 'ORDER', 'UNION', 'WHERE', 'WINDOW', <EOF>", query_id=20260131_095338_00035_mnivk)
[SQL: SELECT *  total_count FROM iceberg.demo_schema.jupyter_users]
(Background on this error at: https://sqlalche.me/e/20/f405)


In [53]:
import pandas as pd
from trino.dbapi import connect

# --- 1. สร้าง Connection ---
# เปรียบเสมือนการโทรศัพท์ไปหา Trino Server ที่กำลังรันอยู่
conn = connect(
    host="docker-compose-trino-coordinator-1",        # หรือ IP เครื่อง Server
    port=8080,               # Port มาตรฐานของ Trino
    user="admin",            # User (ปกติ Trino ไม่เช็ค Password ถ้าไม่ได้ตั้งไว้)
    catalog="iceberg",       # Catalog ที่เราตั้งค่าไว้
    schema="test_db",        # Schema เริ่มต้น (ใส่หรือไม่ใส่ก็ได้)
)

# --- 2. เขียน SQL ---
# ใช้ SQL มาตรฐานได้เลย (ANSI SQL)
sql_query = """
select * from iceberg.demo_schema.jupyter_users

"""

print(f"🚀 Sending Query to Trino: {sql_query}")

# --- 3. ดึงข้อมูลใส่ Pandas DataFrame ---
# วิธีนี้ง่ายที่สุดสำหรับการวิเคราะห์ข้อมูลต่อ
df = pd.read_sql(sql_query, conn)
display(df)
conn.close()

🚀 Sending Query to Trino: 
select * from iceberg.demo_schema.jupyter_users




/tmp/ipykernel_950/187465301.py:25: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,id,username,created_at
0,3,Hive_Master,2026-01-31 08:51:08.506066+00:00
1,4,test_insert,2026-01-31 09:06:22.009201+00:00
2,1,test,2026-01-31 08:51:08.506059+00:00
3,2,Spark_Ninja,2026-01-31 08:51:08.506065+00:00


In [5]:
import prettytable

# ตรวจสอบว่ามีค่า DEFAULT หรือไม่ ถ้าไม่มีให้ยัดค่ามั่วๆ (แต่ใช้งานได้) ใส่กลับเข้าไป
if not hasattr(prettytable, 'DEFAULT'):
    prettytable.DEFAULT = prettytable.MSWORD_FRIENDLY
    
print("Fix applied! You can run SQL now.")

Fix applied! You can run SQL now.


/tmp/ipykernel_8585/1258397917.py:4: DeprecationWarning: the 'DEFAULT' constant is deprecated, use the 'TableStyle' enum instead
  if not hasattr(prettytable, 'DEFAULT'):


In [2]:
!pip uninstall -y ipython-sql
!pip install jupysql


[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [4]:
!pip install --upgrade "grpcio>=1.48.1" google-auth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 21.4 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.00/6 [typing-extensions]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [google-auth] [google-auth]es]

[notice] A new release of pip is available: 25.3 -> 26.0
[notice] To update, run: pip install --upgrade pip


In [5]:
%load_ext sql
%sql trino://admin@docker-compose-trino-coordinator-1:8080/iceberg/test_db


Connecting to 'trino://admin@docker-compose-trino-coordinator-1:8080/iceberg/test_db'

Running query in 'trino://admin@docker-compose-trino-coordinator-1:8080/iceberg/test_db'

id,username,created_at
2,Spark_Ninja,2026-01-31 08:51:08.506065+00:00
3,Hive_Master,2026-01-31 08:51:08.506066+00:00
1,test,2026-01-31 08:51:08.506059+00:00
4,test_insert,2026-01-31 09:06:22.009201+00:00


In [6]:
%sql SELECT * FROM iceberg.demo_schema.jupyter_users

Running query in 'trino://admin@docker-compose-trino-coordinator-1:8080/iceberg/test_db'

id,username,created_at
3,Hive_Master,2026-01-31 08:51:08.506066+00:00
4,test_insert,2026-01-31 09:06:22.009201+00:00
2,Spark_Ninja,2026-01-31 08:51:08.506065+00:00
1,test,2026-01-31 08:51:08.506059+00:00


In [ ]:
import os
from pyspark.sql import SparkSession

# --- 1. CONFIGURATION ---
# ถ้าคุณรัน Jupyter บน Windows/Mac (นอก Docker) ให้ใช้ 'localhost'
# ถ้าคุณรัน Jupyter ใน Docker Network เดียวกัน ให้ใช้ชื่อ Service เช่น 'minio', 'hive-metastore'
MINIO_ENDPOINT = "http://minio:9000"
HIVE_METASTORE_URI = "thrift://docker-compose-hive-metastore-1:9083" 

# Credentials (ต้องตรงกับ docker-compose)
ACCESS_KEY = "minio"
SECRET_KEY = "minio123"

# Packages ที่จำเป็น (Spark 3.5 + Iceberg + AWS S3)
PACKAGES = [
    "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.4.3",
    "org.apache.hadoop:hadoop-aws:3.3.4",
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0" # เผื่อใช้ Kafka
]

# --- 2. BUILD SPARK SESSION ---
spark = (
    SparkSession.builder
    .appName("Jupyter-Iceberg-Hive")
    .master("local[*]") # รันแบบ Local ใช้ทุก CPU Core
    
    # Download Packages
    .config("spark.jars.packages", ",".join(PACKAGES))
    
    # Extensions (สำคัญมากสำหรับ Update/Delete/Merge ใน Iceberg)
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")

    # --- S3 (MinIO) Config ---
    .config("spark.hadoop.fs.s3a.endpoint", MINIO_ENDPOINT)
    .config("spark.hadoop.fs.s3a.access.key", ACCESS_KEY)
    .config("spark.hadoop.fs.s3a.secret.key", SECRET_KEY)
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .config("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
    # Tuning S3
    .config("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem") 

    # --- Iceberg Catalog Config (เชื่อม Hive) ---
    .config("spark.sql.catalog.iceberg", "org.apache.iceberg.spark.SparkCatalog")
    .config("spark.sql.catalog.iceberg.type", "hive")
    .config("spark.sql.catalog.iceberg.uri", HIVE_METASTORE_URI)
    .config("spark.sql.catalog.iceberg.warehouse", "s3a://warehouse/iceberg_data")
    
    .getOrCreate()
)

# ลด Log ขยะ ให้เหลือแค่ Error/Warn
spark.sparkContext.setLogLevel("WARN")

print(f"✅ Spark Session Created! Version: {spark.version}")

In [5]:
try:
    # 1. สร้าง Database (ถ้ายังไม่มี)
    spark.sql("CREATE NAMESPACE IF NOT EXISTS iceberg.demo_schema")
    spark.sql("""
        CREATE TABLE IF NOT EXISTS hive.demo_schema.jupyter_users (
            id INT,
            username STRING,
            created_at TIMESTAMP
        )
        USING iceberg
    """)

    # 3. เตรียมข้อมูล Mock Data (DataFrame)
    from datetime import datetime
    data = [
        (4, "test_insert", datetime.now()),
  
    ]
    df = spark.createDataFrame(data, ["id", "username", "created_at"])

    # 4. เขียนข้อมูลลงตาราง (Append)
    print("Writing data to Iceberg/MinIO...")
    df.writeTo("iceberg.demo_schema.jupyter_users").append()

    # 5. อ่านข้อมูลกลับมาดู
    print("--- 📊 Result Query ---")
    result = spark.sql("SELECT * FROM hive.demo_schema.jupyter_users ORDER BY id DESC")
    result.show()

except Exception as e:
    print(f"❌ Operation Failed: {e}")

❌ Operation Failed: An error occurred while calling o49.sql.
: org.apache.spark.SparkException: [INTERNAL_ERROR] Undefined error message parameter for error class: '_LEGACY_ERROR_TEMP_1055'. Parameters: Map(database -> iceberg.demo_schema)
	at org.apache.spark.SparkException$.internalError(SparkException.scala:92)
	at org.apache.spark.SparkException$.internalError(SparkException.scala:96)
	at org.apache.spark.ErrorClassesJsonReader.getErrorMessage(ErrorClassesJSONReader.scala:56)
	at org.apache.spark.SparkThrowableHelper$.getMessage(SparkThrowableHelper.scala:53)
	at org.apache.spark.SparkThrowableHelper$.getMessage(SparkThrowableHelper.scala:40)
	at org.apache.spark.sql.AnalysisException.<init>(AnalysisException.scala:47)
	at org.apache.spark.sql.AnalysisException.<init>(AnalysisException.scala:70)
	at org.apache.spark.sql.errors.QueryCompilationErrors$.invalidDatabaseNameError(QueryCompilationErrors.scala:876)
	at org.apache.spark.sql.catalyst.analysis.ResolveSessionCatalog$DatabaseN

In [10]:
jdbcUrl = "jdbc:postgresql://docker-compose-postgresdb-1:5432/postgres"
connectionProperties = {
  "user": "postgres",
  "password": "",
  "driver": 'org.postgresql.Driver' # Specify the driver class
}
# df = spark.read.jdbc(jdbcUrl, "public.products", properties=connectionProperties)
# products.write.mode("overwrite") .jdbc(jdbcUrl, "public.products", properties=connectionProperties)

In [9]:
spark.sql("update iceberg.demo_schema.jupyter_users set username ='test' where username='Data_Wizard' ").show()


AnalysisException: [REQUIRES_SINGLE_PART_NAMESPACE] spark_catalog requires a single-part namespace, but got `iceberg`.`demo_schema`.

In [8]:
order_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("order_date", StringType(), True),
    StructField("user_id", StringType(), True),
    StructField("product_id", StringType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("status", StringType(), True),
])
print(order_schema)

StructType([StructField('order_id', StringType(), True), StructField('order_date', StringType(), True), StructField('user_id', StringType(), True), StructField('product_id', StringType(), True), StructField('quantity', IntegerType(), True), StructField('status', StringType(), True)])


In [5]:
orders = spark.read.option('header','true').csv('s3a://lmwn/data/orders.csv').alias('o')
products = spark.read.option('header','true').csv('s3a://lmwn/data/products.csv').alias('p')
orders_pg = spark.read.jdbc(jdbcUrl, "public.orders", properties=connectionProperties)
products_pg = spark.read.jdbc(jdbcUrl, "public.products", properties=connectionProperties)
orders_iceberg = spark.table('iceberg.orders')
products_iceberg = spark.table('iceberg.products')
orders_stream = spark.readStream.format("kafka").option("kafka.bootstrap.servers", "kafka:9092").option("subscribe", "orders").option("startingOffsets", "earliest").load()
order_product = orders.join(products,col('o.product_id')==col('p.product_id'),'left').select( col('order_id'),
                                                                                                  col('order_date'),
                                                                                                  col('user_id'),
                                                                                                  col('o.product_id'),
                                                                                                  col('quantity'),
                                                                                                  col('status'),
                                                                                                  col('product_name'),
                                                                                                  col('price'),
                                                                                                  col('category'))

In [7]:
order_stream = (
    orders_stream.select("timestamp", "value")
      .withColumn("value", from_json(col("value").cast("string"), order_schema)).
                select(col('timestamp'),
                    date_trunc("minute", col("timestamp")).alias("window_start"),
                    col("value.product_id").alias("product_id"),
                    col("value.order_id").alias("order_id"))
)

In [8]:
final_df =  order_stream.withWatermark('timestamp', "1 minutes")\
                .groupBy(col("product_id"),window(col('timestamp'),"1 minutes"))\
                    .agg(count(col("order_id")).alias("order_count")).select(
                        col("product_id"),             
                        col("window.start").alias("window_start"),
                        col("window.end").alias("window_end"),
                        col("order_count")
                    )

In [ ]:
order_product.write.mode("overwrite") .jdbc(jdbcUrl, "public.order_product", properties=connectionProperties)

In [ ]:
order_product.writeTo("iceberg.order_product").append() # .create() #.overwritePartitions()

In [11]:
order_stream.writeStream.outputMode("append")\
    .format("iceberg")\
    .option("checkpointLocation", "s3a://lmwn/checkpoints/orders")\
    .toTable("iceberg.stream.orders_stream")


In [12]:
final_df.writeStream.outputMode("append")\
    .format("iceberg")\
    .option("checkpointLocation", "s3a://lmwn/checkpoints/orders")\
    .toTable("iceberg.stream.orders_log")

In [ ]:
%sql